In [1]:
import pandas as pd

import numpy as np

from datetime import datetime

import requests
from requests.exceptions import RequestException
import json

from datetime import datetime




In [2]:
def get_units_mapping(base_url):
    # Check if the specified conditions are met

    # base_url = "http://localhost:31310/v1/projects"
    page = 1
    limit = 100
    all_projects = []
#
    while True:
        print(page)

        
        url = f"{base_url}?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page={page}&limit={limit}"
        print(url)
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            print(response)
            break

        data = response.json()
        all_projects.extend(data['data'])
        # if page >= 1:
        if page >= data['pageCount']:
            break
        page += 1
    
    projects = all_projects
   
    return pd.DataFrame(all_projects)

base_url = "http://localhost:31310/v1/units"



        

In [18]:
unitsUploaded = get_units_mapping(base_url)

1
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=1&limit=100
2
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=2&limit=100
3
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=3&limit=100
4
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=4&limit=100
5
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=5&limit=100
6
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=6&limit=100
7
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=7&limit=100
8
http://localhost:31310/v1/units?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=8&limit=100
9
http://localho

In [24]:
df_held_1000 = pd.read_excel("./output/held_units_1000.xlsx")
df_retired_1000 = pd.read_excel("./output/retired_units_1000.xlsx")
df_held_2016_all =pd.read_excel("./output/held_units_bef2016_all.xlsx")
df_retired_2016_all=pd.read_excel("./output/retired_units_bef2016all.xlsx")



In [25]:
print(len(df_held_1000))
print(len(df_retired_1000))
print(len(df_held_2016_all))
print(len(df_retired_2016_all))



1701
11531
10994
24357


In [29]:
# Find rows in held units 2016 that are not in held units 1000
df_held_not_in_1000 = pd.merge(df_held_2016_all, df_held_1000, indicator=True, how='outer').query('_merge == "left_only"').drop('_merge', axis=1)

# Find rows in retired units 2016 that are not in retired units 1000
df_retired_not_in_1000 = pd.merge(df_retired_2016_all, df_retired_1000, indicator=True, how='outer').query('_merge == "left_only"').drop('_merge', axis=1)

print(len(df_held_not_in_1000))
print(len(df_retired_not_in_1000))

print(len(df_held_not_in_1000)+len(df_held_1000))
print(len(df_retired_not_in_1000)+len(df_retired_1000))





9293
12826
10994
24357


In [41]:
df_held_not_in_1000.to_excel("cleaned_bef_2016_held.xlsx")
df_retired_not_in_1000.to_excel("cleaned_bef_2016_retired.xlsx")

In [19]:
print(unitsUploaded.columns)
unitsUploaded.to_excel("./output/unitsCDMUploaded.xlsx")
grouped_df = unitsUploaded[unitsUploaded.duplicated(subset=['issuanceId','serialNumberBlock'], keep=False)]
grouped_df = grouped_df.sort_values(['issuanceId','serialNumberBlock'])
grouped_df.to_excel("units_duplicate_mistake.xlsx")



Index(['warehouseUnitId', 'issuanceId', 'projectLocationId', 'orgUid',
       'unitOwner', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'serialNumberBlock', 'unitBlockStart',
       'unitBlockEnd', 'unitCount', 'vintageYear', 'unitType', 'marketplace',
       'marketplaceLink', 'marketplaceIdentifier', 'unitTags', 'unitStatus',
       'unitStatusReason', 'unitRegistryLink',
       'correspondingAdjustmentDeclaration', 'correspondingAdjustmentStatus',
       'timeStaged', 'createdAt', 'updatedAt', 'labels', 'issuance'],
      dtype='object')


In [22]:
# grouped_df.unique(['issuanceId','serialNumberBlock'])
unique_df = grouped_df.drop_duplicates(['issuanceId','serialNumberBlock','vintageYear'])

print(len(unique_df))
duplicate_rows = grouped_df[grouped_df.duplicated(subset=['issuanceId','serialNumberBlock','vintageYear'], keep=False)]

print(len(duplicate_rows))


unique_mask = ~grouped_df.duplicated(subset=['issuanceId','serialNumberBlock','vintageYear'], keep='first')

# The unique DataFrame
# unique_df = grouped_df[unique_mask]

# Step 2: Get rows that are not part of the unique DataFrame
# We find the rows in df which are not in unique_df by using an index based approach
duplicates_df = grouped_df[~unique_mask]

duplicates_df

7258
17497


,warehouseUnitId,issuanceId,projectLocationId,orgUid,unitOwner,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,serialNumberBlock,unitBlockStart,unitBlockEnd,...,unitStatus,unitStatusReason,unitRegistryLink,correspondingAdjustmentDeclaration,correspondingAdjustmentStatus,timeStaged,createdAt,updatedAt,labels,issuance
42721,d7741e10-eb52-4214-a0c0-e3b28bcacbb9,01764c8e-d838-4803-a87e-023ee9047bfd,b4cbc6cd-4281-4b6b-8b37-879cad3735fc,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,China,China,China,897975888.0-898129048,897975888.0,898129048,...,Held,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704442196,2024-01-05T09:31:56.088Z,2024-01-05T09:31:56.088Z,[],"{'id': '01764c8e-d838-4803-a87e-023ee9047bfd',..."
42669,9339ec3e-d8fd-4f6e-bd5c-4fd54ad29c2b,01bc7097-a157-44ee-877c-2e586bc97bcb,3c5c1114-17e3-4fb6-94f6-5fbab6e34e24,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,India,India,India,251028190.0-251148493,251028190.0,251148493,...,Held,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704442407,2024-01-05T09:31:58.637Z,2024-01-05T09:31:58.637Z,[],"{'id': '01bc7097-a157-44ee-877c-2e586bc97bcb',..."
45568,99548d7f-e34f-4d0d-9d36-e67fab7bb5d8,01bc7097-a157-44ee-877c-2e586bc97bcb,3c5c1114-17e3-4fb6-94f6-5fbab6e34e24,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,India,India,India,251028190.0-251148493,251028190.0,251148493,...,Held,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704301647,2024-01-03T17:22:26.434Z,2024-01-03T17:22:26.434Z,[],"{'id': '01bc7097-a157-44ee-877c-2e586bc97bcb',..."
22120,98d8d6be-b9ad-4b18-a60f-215790f19428,01d47a5b-7286-4219-b12f-a1ee23b4a108,84e0dfdc-27d5-411a-83b5-d65c420dfc08,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,Chile,Chile,Chile,10418149-10418162,10418149,10418162,...,Retired,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1711378388,2024-03-25T16:44:41.748Z,2024-03-25T16:44:41.748Z,[],"{'id': '01d47a5b-7286-4219-b12f-a1ee23b4a108',..."
42374,1d6b32ff-c042-46a0-af97-f7ff5161864d,02adb3ce-557a-4466-ac1b-9846ecd68b09,81de7d9d-2f81-46b8-ace9-37fc8b0f49fd,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,Brazil,Brazil,Brazil,169731719.0-169745681,169731719.0,169745681,...,Held,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704444132,2024-01-05T09:31:54.794Z,2024-01-05T09:31:54.794Z,[],"{'id': '02adb3ce-557a-4466-ac1b-9846ecd68b09',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41687,5c744ee0-92d5-4eb2-9630-f0e403ea5518,fd69c9ce-6a1b-448d-b774-c544fd9e8ded,3d6088c5-a2da-4c7b-a09a-c109676d592a,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,Colombia,Colombia,Colombia,21054900-21056620,21054900,21056620,...,Retired,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704458954,2024-01-05T19:09:13.232Z,2024-01-05T19:09:13.232Z,[],"{'id': 'fd69c9ce-6a1b-448d-b774-c544fd9e8ded',..."
41555,bfce1685-37c0-4f2c-b215-13cf2d6e4390,fd69c9ce-6a1b-448d-b774-c544fd9e8ded,3d6088c5-a2da-4c7b-a09a-c109676d592a,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,Colombia,Colombia,Colombia,21056621-21070920,21056621,21070920,...,Retired,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704478733,2024-01-05T19:09:17.509Z,2024-01-05T19:09:17.509Z,[],"{'id': 'fd69c9ce-6a1b-448d-b774-c544fd9e8ded',..."
42328,9aaecdeb-6a6f-4992-819a-2fbcdb748586,fd69c9ce-6a1b-448d-b774-c544fd9e8ded,3d6088c5-a2da-4c7b-a09a-c109676d592a,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,Colombia,Colombia,Colombia,21070197.0-21090305,21070197.0,21090305,...,Held,None,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Applicable,1704444317,2024-01-05T09:31:54.181Z,2024-01-05T09:31:54.181Z,[],"{'id': 'fd69c9ce-6a1b-448d-b774-c544fd9e8ded',..."
42176,eecb662f-ac95-4f89-989a-4e298d05809f,fd762051-50e0-46b9-a8aa-e7d87095b35f,d5d013e0-2998-4e53-a290-fdc0f667f647,b3

In [23]:
len(unitsUploaded)



50524

In [4]:
duplicates_df = pd.read_excel("units_duplicate_mistake.xlsx")


In [5]:
import requests
import pandas as pd

duplicates_df['warehouseUnitId']


def delete_unit(df, column_name,start,end):
    """
    Send a DELETE request for each unique warehouse unit ID from the specified DataFrame column.

    Parameters:
    df (pd.DataFrame): DataFrame containing the warehouse unit IDs.
    column_name (str): The column name where the warehouse unit IDs are stored.

    Returns:
    dict: A dictionary with warehouse unit IDs and their corresponding deletion status.
    """
    results = {}
    base_url = 'http://localhost:31310/v1/units'
    headers = {'Content-Type': 'application/json'}
    
    # Ensure unique IDs to avoid redundant requests
    slice_df = df.iloc[start:end]
    unique_ids = slice_df[column_name]

    i=start
    for unit_id in unique_ids:
        print(i)
        data = {"warehouseUnitId": unit_id}
        response = requests.delete(base_url, json=data, headers=headers)

        # Record the status of each delete operation
        results[unit_id] = {
            'status_code': response.status_code,
            'response': response.json() if response.status_code == 200 else response.text
        }
        i+=1
    return results


# results = delete_unit(duplicates_df, 'warehouseUnitId',10,100)
# print(results)


In [9]:

results = delete_unit(duplicates_df, 'warehouseUnitId',7000,8500)

7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
